# RAG con SQL (Text-to-SQL)

Configuración inicial: carga de variables de entorno y cliente OpenAI.

In [ ]:
import sqlite3
import os
from pathlib import Path
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv(override=True)

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"]
)
CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_DEPLOYMENT_CHAT"]

print(f"Usando modelo: {CHAT_DEPLOYMENT}")

Conexión a la base de datos SQLite y extracción del esquema de la tabla para informar al LLM.

In [ ]:
# Connect to database
# We check relative path assuming notebook is in 02-sql-rag/
db_path = Path("data/hybrid_cars.db")
if not db_path.exists():
    # Fallback for absolute path if running from root context
    db_path = Path("02-sql-rag/data/hybrid_cars.db")

print(f"Usando base de datos: {db_path.absolute()}")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Get database schema for the LLM
cursor.execute("PRAGMA table_info(cars)")
schema_info = cursor.fetchall()
schema_description = "Tabla: cars\nColumnas:\n"
for col in schema_info:
    schema_description += f"  - {col[1]} ({col[2]}): "
    if col[1] == "vehicle":
        schema_description += "Nombre del modelo de coche"
    elif col[1] == "year":
        schema_description += "Año de fabricación"
    elif col[1] == "msrp":
        schema_description += "Precio de venta sugerido por el fabricante en dólares"
    elif col[1] == "acceleration":
        schema_description += "Tiempo 0-60 mph en segundos (menor es más rápido)"
    elif col[1] == "mpg":
        schema_description += "Millas por galón (mayor es más eficiente)"
    elif col[1] == "class":
        schema_description += "Clase de vehículo (Compacto, Mediano, SUV, etc.)"
    schema_description += "\n"

print("\nEsquema de la base de datos:")
print(schema_description)

Definición de los mensajes del sistema (prompts) para la generación de SQL y la respuesta en lenguaje natural.

In [ ]:
# System message for SQL generation
SQL_GENERATION_SYSTEM_MESSAGE = f"""
Eres un asistente experto en SQL. Tu tarea es convertir preguntas en lenguaje natural en consultas SQLite válidas.

Esquema de la base de datos:
{schema_description}

Reglas importantes:
1. Genera SOLO la consulta SQL, sin explicaciones ni formato markdown
2. Usa sintaxis SQLite apropiada
3. Usa siempre sentencias SELECT (no INSERT, UPDATE, DELETE, DROP)
4. Usa cláusulas WHERE, ORDER BY, LIMIT según sea necesario
5. Para preguntas sobre "más rápido", usa ORDER BY acceleration ASC (menor es más rápido)
6. Para preguntas sobre "más lento", usa ORDER BY acceleration DESC
7. Para "más eficiente" o "mejor mpg", usa ORDER BY mpg DESC
8. Para "más barato", usa ORDER BY msrp ASC
9. Para "más caro", usa ORDER BY msrp DESC
10. Si se pregunta por un resultado único o "el", añade LIMIT 1
11. Usa LIKE para coincidencias parciales (ej., WHERE vehicle LIKE '%Prius%')
12. Considera el contexto de la conversación para refinar las consultas

Ejemplos:
P: "¿qué tan rápido es el prius v?"
R: SELECT vehicle, acceleration, year, mpg FROM cars WHERE vehicle LIKE '%Prius V%'

P: "¿cuál es el coche más rápido?"
R: SELECT vehicle, acceleration, year, msrp FROM cars ORDER BY acceleration ASC LIMIT 1

P: "muéstrame coches eficientes"
R: SELECT vehicle, mpg, year, class FROM cars WHERE mpg > 40 ORDER BY mpg DESC

P: "dame la pickup más barata"
R: SELECT vehicle, msrp, year, mpg FROM cars WHERE class = 'Pickup Truck' ORDER BY msrp ASC LIMIT 1
"""

ANSWER_SYSTEM_MESSAGE = """
Eres un asistente útil que responde preguntas sobre coches basándote en una base de datos de coches híbridos.
Debes usar los datos de los resultados de la consulta para responder las preguntas con precisión.
Sé conciso y directo en tus respuestas. Usa el historial de la conversación para proporcionar respuestas contextuales.
"""

Función principal que orquesta el flujo: pregunta -> SQL -> ejecución -> respuesta natural.

In [ ]:
def process_question(question, chat_history=None):
    if chat_history is None:
        chat_history = [{"role": "system", "content": ANSWER_SYSTEM_MESSAGE}]
    
    print(f"Pregunta: {question}")
    
    try:
        # Generate SQL query using the LLM with conversation context
        sql_response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT,
            temperature=0.1,
            messages=[
                {"role": "system", "content": SQL_GENERATION_SYSTEM_MESSAGE},
                {
                    "role": "user",
                    "content": f"Nueva pregunta: {question}\n\nContexto de conversación: {chat_history[-4:] if len(chat_history) > 4 else chat_history}",
                },
            ],
        )

        sql_query = sql_response.choices[0].message.content.strip()
        # Remove any markdown code blocks if present
        if sql_query.startswith("```"):
            sql_query = sql_query.split("\n", 1)[1]
            sql_query = sql_query.rsplit("```", 1)[0]
        sql_query = sql_query.strip()

        print(f"\nSQL Generado: {sql_query}")

        # Execute the query
        cursor.execute(sql_query)
        results = cursor.fetchall()

        # Get column names
        column_names = [description[0] for description in cursor.description]

        # Format results as markdown table
        if results:
            results_table = " | ".join(column_names) + "\n"
            results_table += " | ".join("---" for _ in column_names) + "\n"
            for row in results[:10]:  # Limit to 10 rows for display
                results_table += " | ".join(str(val) if val is not None else "N/A" for val in row) + "\n"

            if len(results) > 10:
                results_table += f"\n... y {len(results) - 10} filas más"

            print(f"\nResultados de la consulta ({len(results)} filas):")
            print(results_table)
        else:
            results_table = "No se encontraron resultados."
            print(f"\n{results_table}")

        # Use the results to generate a natural language response
        chat_history.append({"role": "user", "content": f"{question}\n\nResultados de la consulta:\n{results_table}"})

        response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT, temperature=0.3, messages=chat_history
        )

        llm_response = response.choices[0].message.content
        chat_history.append({"role": "assistant", "content": llm_response})

        print(f"\nRespuesta:")
        print(llm_response)
        
        return chat_history

    except sqlite3.Error as e:
        print(f"\nError SQL: {e}")
        print("La consulta generada podría ser inválida.")
        return chat_history
    except Exception as e:
        print(f"\nError: {e}")
        return chat_history

Inicialización del historial de conversación y ejecución de ejemplos de preguntas.

In [ ]:
# Initialize conversation history
history = [{"role": "system", "content": ANSWER_SYSTEM_MESSAGE}]

In [ ]:
# Prompt 1
history = process_question("¿cuál es el coche más caro de cada año?", history)

In [ ]:
# Prompt 2
history = process_question("¿Qué coche tiene la mejor aceleración (menor tiempo 0-60)?", history)

In [ ]:
# Prompt 3
history = process_question("Muéstrame los 3 SUVs más eficientes en consumo", history)